<a href="https://colab.research.google.com/github/rahiakela/building-computer-vision-applications-using-artificial-neural-networks/blob/master/6-deep-learning-in-object-detection/3_training_YOLOv3_model_for_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a YOLOv3 Model for Object Detection

YOLOv3 is the youngest of all the object detection algorithms.It has not made it to the TensorFlow object detection API yet.YOLOv3 uses the Darknet-53 architecture to train the model.

We will use the official API and weights of the pretrained model to perform transfer learning of our YOLOv3 model from the same Oxford-IIIT Pet dataset that we used in the previous SSD model. We will run the training on Google Colab and use a GPU hardware accelerator.

## Installing the Darknet Framework

Darknet is an open source neural network framework written in C and CUDA that runs on both CPUs and GPUs. First, clone the Darknet GitHub repository and then build the source.

In [1]:
%%shell

git clone https://github.com/ansarisam/darknet.git
# Official repository
#git clone https://github.com/pjreddie/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 5912, done.
remote: Total 5912 (delta 0), reused 0 (delta 0), pack-reused 5912
Receiving objects: 100% (5912/5912), 6.34 MiB | 18.50 MiB/s, done.
Resolving deltas: 100% (3919/3919), done.


After the repository is cloned, expand the file browser, navigate to the darknet
directory, and download the Makefile to your local computer. Edit the Makefile
(highlighted in bold letters) and change GPU=1 and OPENCV=1, as shown here:

```c
GPU=1
CUDNN=0
OPENCV=1
OPENMP=0
DEBUG=0
```

Make sure no other change is made to the Makefile, or you may have trouble
building your Darknet code.

Now we are ready to build the Darknet framework.

In [ ]:
# Running the make Command to Build Darknet
%%shell
cd darknet/
make

After the build process successfully completes, run the below command to test your installation. 

It should print `usage: ./darknet <function>` if the installation is successful.

In [3]:
# Testing the Darknet Installation
%%shell
cd darknet
./darknet

usage: ./darknet <function>


## Downloading Pre-trained Convolutional Weights

Let's downloads pre-trained weights of the COCO dataset trained on the
Darknet-53 framework.

In [4]:
# Downloading Pre-trained Darknet-53 Weights
%%shell
mkdir pretrained
cd pretrained
wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-11-30 10:01:06--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   161KB/s    in 12m 35s 

2020-11-30 10:13:42 (210 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



## Downloading an Annotated Oxford-IIIT Pet Dataset

Let's downloads the pet dataset with both the images and annotations.

In [ ]:
# Downloading the Pet Dataset Images and Annotations
%%shell

mkdir petdata
cd petdata

wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

tar -xvf images.tar.gz
tar -xvf annotations.tar.gz

The images directory contains a few files with the extension .mat, which
causes the training to break. So need to remove these .mat files.

In [6]:
# Deleting the Invalid File Extension .mat
%%shell

cd /content/petdata/images
rm *.mat

## Preparing the Dataset

The YOLOv3 training API expects the dataset to have a certain format and directory structure. The pet data that we downloaded has two subdirectories:

- **images** and 
- **annotations**. 

The images directory contains all the labeled images that we will use for
training and testing. The annotations directory contains annotation files in XML format, one XML file per image.

YOLOv3 expects the following files:

- **train.txt**: This file contains the absolute path of images—one image path
per line—that will be used for training.
- **test.txt**: This file contains the absolute path of images—one image path
per line—that will be used for testing.
- **class.data**: This file contains a list of names of the object classes—one
name per line.
- **labels**: This directory is in the same location where train.txt and test.txt are located. This labels directory contains annotation files, one file per image. The file name in this directory must be the same as the image file name, except that it has the extension .txt.

For example, if the image file name is `Abyssinian_1.jpg`, the annotation file name in the labels directory must be `Abyssinian_1.txt`. Each annotation text file must contain the annotated bounding box and object class in one single line in the following format:

```python
<object-class> <x_center> <y_center> <width> <height>
```

where
- `<object-class>` is the integer class index of the object, from 0 to (num_
class-1).
- `<x_center> and <y_center>` are float values representing the center of
the bounding boxes relative to the image height and width.
- `<width> <height>` are the width and height of bounding boxes relative
to the image height and width.

Note that the entries in this file are separated by blank spaces and not
by commas or any other delimiters.

An example entry of the annotation text file is as follows (ensure the fields are separated by white space and not comma or any other delimiter.):

```python
10 0.63 0.28500000000000003 0.28500000000000003 0.215
```

In [ ]:
# Converting Image Annotations from XML to TXT
